1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv 

DT = pd.read_csv('data/data_000637.txt').head(12000) 
print(DT) 
print(DT.info())

FileNotFoundError: [Errno 2] File b'data/data_000637.txt' does not exist: b'data/data_000637.txt'

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [2]:
NT= data["BX_COUNTER"]
est_NT = max(NT)
print("Estimated number of BX in a ORBIT:", est_NT)

NameError: name 'data' is not defined

2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [4]:
NT= data["BX_COUNTER"]
est_NT = max(NT)
print("Estimated number of BX in a ORBIT:", est_NT)

NameError: name 'data' is not defined

3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [ ]:
data["abs_time_in_ns"] = data['TDC_MEAS'].transform(lambda x: x * 25/30) + data['BX_COUNTER'].transform(lambda x: x * 25) +data['ORBIT_CNT'].transform(lambda x: x * est_bx * 25)
data

4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [5]:
import datetime as dt
itime = dt.datetime.now()
print("Begin time:", itime)
time =  data['TDC_MEAS'] *(25/30) + data['BX_COUNTER'] * 25 + data['ORBIT_CNT']*est_bx*25
ftime = dt.datetime.now()
print("End time:", ftime)
print("Elapsed time:", (ftime - itime))

Begin time: 2022-12-02 21:14:31.131481


NameError: name 'data' is not defined

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [ ]:
grouped =  data.groupby('TDC_CHANNEL').sum().sort_values(by = ['HEAD']).iloc[-3:]
grouped

6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [6]:
count = data.ORBIT_CNT.unique().size
print("The number of non empty orbits is: " + str(count))

NameError: name 'data' is not defined

7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [ ]:
L=139.
unique_orbits = pd.DataFrame(data[data['TDC_CHANNEL'] == 139]).drop_duplicates(subset=['ORBIT_CNT'], inplace=False)
print("\nThe number of unique orbits with at least one measurement from TDC_CHANNEL == 139 is:", len(unique_orbits))

8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [ ]:
FPGA_0 = pd.Series(data[data['FPGA'] == 0]['TDC_CHANNEL'].value_counts())
FPGA_1 = pd.Series(data[data['FPGA'] == 1]['TDC_CHANNEL'].value_counts())
print("Series FPGA == 0 :\n" , FPGA_0)
print("\nSeries FPGA == 1 :\n" , FPGA_1)

9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.